In [1]:
import pandas as pd
!conda install -c anaconda lxml --yes
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np # library to handle data in a vectorized manner
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
import json # library to handle JSON files

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.1.1   |                0         132 KB  anaconda
    certifi-2019.11.28         |           py36_0         156 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    lxml-4.5.0                 |   py36hefd8a0e_0         1.6 MB  anaconda
    ------------------------------------------------------------
                                           Total:         6.9 MB

The following packages will be UPDATED:

    ca-certificates: 2019.11.27-0         --> 2020.1.1-0           anaconda
    certifi:         2019.11.28-py36_0    --> 2019.11.28-py36_0    anaconda
    lxml:            4.3.1-py36hefd8a0e_0 --> 4.5.0-py36hefd8a0e_0 anaconda
    openssl:        

In [4]:
# Scrapping wikipedia page using pandas
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.drop(df.loc[df['Borough']=='Not assigned'].index, inplace=True)
df.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
9,M9A,Queen's Park,Not assigned
10,M1B,Scarborough,Rouge
11,M1B,Scarborough,Malvern
13,M3B,North York,Don Mills North


In [3]:
df.shape

(210, 3)

In [5]:
geolocator = Nominatim(user_agent="Torrento_explorer")

In [6]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['Borough'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [7]:
df.head()

,Postcode,Borough,Neighbourhood,location,point,latitude,longitude,altitude
2,M3A,North York,Parkwoods,"(North York, Toronto, Golden Horseshoe, Ontari...","(43.7543263, -79.44911696639593, 0.0)",43.754326,-79.449117,0.0
3,M4A,North York,Victoria Village,"(North York, Toronto, Golden Horseshoe, Ontari...","(43.7543263, -79.44911696639593, 0.0)",43.754326,-79.449117,0.0
4,M5A,Downtown Toronto,Harbourfront,"(CF Toronto Eaton Centre, 220, Yonge Street, D...","(43.6541737, -79.38081164513409, 0.0)",43.654174,-79.380812,0.0
5,M6A,North York,Lawrence Heights,"(North York, Toronto, Golden Horseshoe, Ontari...","(43.7543263, -79.44911696639593, 0.0)",43.754326,-79.449117,0.0
6,M6A,North York,Lawrence Manor,"(North York, Toronto, Golden Horseshoe, Ontari...","(43.7543263, -79.44911696639593, 0.0)",43.754326,-79.449117,0.0


In [8]:
df = df.drop(['location','point','altitude'],axis=1)
df

,Postcode,Borough,Neighbourhood,latitude,longitude
2,M3A,North York,Parkwoods,43.754326,-79.449117
3,M4A,North York,Victoria Village,43.754326,-79.449117
4,M5A,Downtown Toronto,Harbourfront,43.654174,-79.380812
5,M6A,North York,Lawrence Heights,43.754326,-79.449117
6,M6A,North York,Lawrence Manor,43.754326,-79.449117
7,M7A,Downtown Toronto,Queen's Park,43.654174,-79.380812
9,M9A,Queen's Park,Not assigned,43.659659,-79.390340
10,M1B,Scarborough,Rouge,54.282001,-0.401187
11,M1B,Scarborough,Malvern,54.282001,-0.401187
13,M3B,North York,Don Mills North,43.754326,-79.449117
